# Project Summary
#### There are many Machine Learning (ML) and Natural Language Processing (NLP) tools to explore. Here I will define several products and analysis goals to achieve with NLP.
* 

In [1]:
%load_ext lab_black

In [2]:
%matplotlib inline

In [1]:
import numpy as np

# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers
# import nltk
# from nltk.corpus import stopwords
import pandas as pd
import re
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
import plotly
import plotly.express as px
import plotly.graph_objects as go
import json
from flask import Flask, jsonify, render_template
from datetime import date


import spacy




ModuleNotFoundError: No module named 'spacy'

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
import pywhatkit

In [ ]:
pywhatkit.text_to_handwriting("""Test this library""")

In [ ]:
# from fpdf import FPDF

## Begin with FOTR

In [ ]:
# Get the data
# Decision: read from a PDF or .txt file? The PDF may require extra data wrangling and cleanup, which is good, but the .txt can be had from Kaggle. Perhaps better practice from PDF.
# Could use both at first to compare...

In [ ]:
PATH = os.path.join("data", "01 - The Fellowship Of The Ring.txt")

In [ ]:
with open(
    PATH, "r", encoding="ISO-8859-1"
) as file:  # changed encoding to deal with "UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf3 in position 2836: invalid continuation byte" error
    lotr_text = file.read()

## Experiment with TensorFlow and Keras
#### Think about the end goal and then plot next steps... this might not be the path to go on since this is specific to the Character RNN exercise in the HOML text.

In [ ]:
# tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)

In [ ]:
tokenizer.fit_on_texts([lotr_text])

In [ ]:
max_id = len(tokenizer.word_index)

In [ ]:
max_id

In [ ]:
dataset_size = tokenizer.document_count

In [ ]:
dataset_size

In [ ]:
[encoded] = np.array(tokenizer.texts_to_sequences([lotr_text])) - 1

### From news-sentiment-analysis-app

In [ ]:
nltk.download("stopwords")
stoplist = stopwords.words("english")

In [ ]:
# Get nGrams: (2, 2) for bigrams, (3, 3) for trigrams...
vectorizer = CountVectorizer(
    stop_words=stoplist, ngram_range=(3, 3)
)  # Converts a collection of text documents to a matrix of token counts: the occurrences of tokens in each document. This implementation produces a sparse representation of the counts.
X = vectorizer.fit_transform([lotr_text])
features = vectorizer.get_feature_names()
print("\n\nX : \n", X.toarray())

In [ ]:
# Getting top ranking features
sums = X.sum(axis=0)
data = []
for col, term in enumerate(features):
    data.append((term, sums[0, col]))
ranking = pd.DataFrame(data, columns=["phrase", "rank"])
words = ranking.sort_values("rank", ascending=False)
print("\n\nWords : \n", words.head(20))

In [ ]:
# Select top 50 nGrams and add to new dataframe
trigram_df = words.head(n=50)

In [ ]:
df_data = trigram_df
term = df_data["phrase"]
frequency = df_data["rank"]

trace1 = {
    "x": term,
    "y": frequency,
    "mode": "markers",
    "hovertemplate": "Trigram: %{x}<br>Count: %{y}<extra></extra>",
    "marker": {
        "color": frequency,
        "size": frequency,
        "sizeref": 0.3,
        "sizemode": "area",
        "opacity": 1,
    },
}

data_to_plot = [
    trace1,
]

plot_layout = {
    "title": "Trigram frequency",
    "autosize": False,
    "height": 700,
    "width": 1200,
    "margin": {"l": 50, "r": 50, "b": 200, "t": 100, "pad": 4},
    "xaxis": {
        "title": "Trigrams",
        "automargin": True,
        "tickangle": 45,
        "titlefont": {"family": "Arial, bold", "size": 18, "color": "black"},
    },
    "yaxis": {
        "title": "Count",
        "automargin": True,
        "type": "log",
        "titlefont": {"family": "Arial, sans-serif", "size": 18, "color": "black"},
    },
}

In [ ]:
fig = go.Figure(data_to_plot, plot_layout)

In [ ]:
fig.update_yaxes(type="log")

In [ ]:
fig.show()